In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Explor the data

In [ ]:
from datasets import load_dataset

ds = load_dataset("wikimedia/wikipedia", "20231101.ar")

Generating train split:   0%|          | 0/1219201 [00:00<?, ? examples/s]

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 1219201
    })
})

In [ ]:
# Extract text from the dataset
for article in ds['train']:
    text = article['text']
    title = article['title']
    print(f"Title: {title}\nText: {text[:500]}...\n")  # Print first 500 characters of each article

Streaming output truncated to the last 5000 lines.

Title: سوين الثالث ملك الدنمارك
Text: سوين الثالث غراتي (بالدنماركية: Svend III Grathe) (حوالي 1125 - 23 أكتوبر 1157) كان ملك الدنمارك بين عامي 1146 و1157، في تحالفات متغيرة مع كانوت الخامس وابن عمه فالديمار الأول. في عام 1157، اتفق الثلاثة على تقسيم الدنمارك الثلاثي حاول سوين قتل منافسيه في مأدبة السلام، لكنه هزم لاحقًا على يد فالديمار الأول في معركة غراث هيث وقتل أثناء محاولته الفرار.

المراجع 

ملوك قتلوا في عمليات قتالية
بيت إستريدسن
مواليد القرن 12
وفيات 1157
ملوك الدنمارك القرن 12...

Title: كيربي ريد
Text: كيربي ريد هي إحدى الشخصيات الخيالية في سلسلة أفلام الصرخة أو سكريم، التي أنشأها كيفن ويليامسون وقامت بأداء دورها هايدن بانيتير. ظهرت لأول مرة في فيلم سكريم 4 لعام (2011) وعادت للظهور مرة أخرى في سكريم 6 (2023).

فيلم سكريم 4 
ظهرت ريد في سكريم 4، كأفضل صديق لابن عم بطل الرواية سيدني بريسكوت الشاب، جيل روبرتس، وهو هدف ألفي لقاتل متسلسل مقلد يرتدي زي جوست فيس سيئ السمعة. كما وصفت كيربي بأنها "الشخصية المفضلة لدى المعجبين"   وال

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
    

In [ ]:


!pip install blingfire



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 12.4 MB/s eta 0:00:00


# import libraries



In [ ]:
import os
import time
import re
import gzip
import gc    # Garbage collector
from tqdm.auto import tqdm
from concurrent.futures import ProcessPoolExecutor, as_completed

import blingfire as bf
import numpy as np
from datasets import load_dataset

# preprocess wikipedia articles and donwload it in txt files

In [ ]:
# Compile regular expressions only once at the beginning
infobox_pattern = re.compile(r'\{\{Infobox [^}]+\}\}', flags=re.DOTALL)
sidebar_pattern = re.compile(r'\{\{Sidebar [^}]+\}\}', flags=re.DOTALL)
link_pattern = re.compile(r'\[\[([^|\]]+\|)?([^\]]+)\]\]')
references_pattern = re.compile(r'==\s*(References|External links|See also|Notes)\s*==.*', flags=re.DOTALL)
citation_needed_pattern = re.compile(r'\{\{citation needed[^}]*\}\}', flags=re.DOTALL)
cn_pattern = re.compile(r'\{\{cn\}\}', flags=re.DOTALL)
curly_braces_pattern = re.compile(r'\{\{[^}]+\}\}', flags=re.DOTALL)
whitespace_pattern = re.compile(r'\s+')

In [ ]:
def preprocess_article(text: str) -> str:
    # Remove infoboxes and sidebars
    text = infobox_pattern.sub('', text)
    text = sidebar_pattern.sub('', text)

    # Simplify links - keep the text of the link only
    text = link_pattern.sub(r'\2', text)

    # Remove sections that start with == References ==, == External links ==, etc.
    text = references_pattern.sub('', text)

    # Optional: Remove citation needed and similar templates
    text = citation_needed_pattern.sub('', text)
    text = cn_pattern.sub('', text)  # Short form of citation needed

    # Remove any remaining curly braces content (catch-all for other templates)
    text = curly_braces_pattern.sub('', text)

    # Normalize whitespace to a single space
    text = whitespace_pattern.sub(' ', text).strip()

    return text

def process_article(article_text: str, min_len: int, max_len: int) -> str:
    # Preprocess the article text with regex
    article_text = preprocess_article(article_text)

    # Skip the processing if there is no relavant sentences
    if not article_text:
        return ""

    # Initialize an empty list to hold sentences that meet the length criteria.
    proper_sentences = []

    # Segment the preprocessed article text into sentences and obtain their offsets (start and end positions).
    _, offsets = bf.text_to_sentences_and_offsets(article_text)

    for o in offsets:
        # Check if the length of the current sentence (calculated as end position - start position)
        # falls within the specified minimum and maximum length bounds.
        if not min_len <= o[1] - o[0] <= max_len:
            # If the sentence does not meet the length criteria, skip to the next iteration.
            continue

        # Extract the sentence from the article text using the start and end positions from 'o'.
        sentence = article_text[o[0]:o[1]]

        # Add the sentence that meets the length criteria to the list of proper sentences.
        proper_sentences.append(sentence)

    # Join the proper sentences into a single string, separated by newline characters,
    # and return this string. This results in a string where each sentence is on a new line,
    # assuming it met the length criteria.
    return '\n'.join(proper_sentences)

def process_article_wrapper(args):
    return process_article(*args)

def process_wikipedia_dataset(wiki_dataset, output_dir, articles_per_file=1_000_000, batch_size=100):
    os.makedirs(output_dir, exist_ok=True)  # Ensure output directory exists
    file_count = 1
    article_count = 0
    out_f = gzip.open(f'{output_dir}/wikipedia_processed_{file_count}.txt.gz', 'wt', encoding='utf-8')

    with ProcessPoolExecutor() as executor:
        futures = {}
        for article in tqdm(wiki_dataset, desc='Processing Articles'):
            # Submit tasks as you iterate
            future = executor.submit(process_article_wrapper, (article['text'], 32, 2048))
            futures[future] = article['text']

            # Process completed tasks in batches to save memory
            if len(futures) >= batch_size:
                for future in as_completed(futures):
                    sentences = future.result()
                    out_f.write(sentences + '\n')
                    article_count += 1

                    if article_count >= articles_per_file:
                        out_f.close()
                        file_count += 1
                        article_count = 0
                        out_f = gzip.open(f'{output_dir}/wikipedia_processed_{file_count}.txt.gz', 'wt', encoding='utf-8')
                    # Remove the future from the dictionary once processed
                    del futures[future]
                    break  # Break after processing one to check if more tasks should be added

        # Process any remaining tasks
        for future in as_completed(futures):
            sentences = future.result()
            out_f.write(sentences + '\n')
            article_count += 1

            if article_count >= articles_per_file:
                out_f.close()
                file_count += 1
                article_count = 0
                out_f = gzip.open(f'{output_dir}/wikipedia_processed_{file_count}.txt.gz', 'wt', encoding='utf-8')

    # Ensure the last file is closed
    out_f.close()

# load the data

In [ ]:


# Load the Wikipedia dataset
wiki_dataset = load_dataset("wikimedia/wikipedia", "20231101.ar", split='train')

print(f'Length of the Wikipedia dataset is {len(wiki_dataset):_} articles.')

# Specify the working directory to save the processed files
output_dir = '/content/sample_data'
# Process the Wikipedia dataset add them to .txt.gz files
process_wikipedia_dataset(wiki_dataset, output_dir)



In [ ]:

# Load the Wikipedia dataset
wiki_dataset = load_dataset("wikimedia/wikipedia", "20231101.ar", split='train')

print(f'Length of the Wikipedia dataset is {len(wiki_dataset):_} articles.')

# Specify the working directory to save the processed files
output_dir = '/content/drive/MyDrive/Rag Dataset'
# Process the Wikipedia dataset add them to .txt.gz files
process_wikipedia_dataset(wiki_dataset, output_dir)

Length of the Wikipedia dataset is 1_219_201 articles.


Processing Articles:   0%|          | 0/1219201 [00:00<?, ?it/s]